## ASG5
          

Cluster the business sections (/blue/acg7849/share/BS) using Doc2Vec (50 clusters) in two ways:

Using a counter as the ‘tag’ (as in 5.1.5)

Using a counter as the ‘tag’, and the industry code as an additional tag (yield TaggedDocument(words=file_tokens, tags=[i, SIC]) where SIC is a string holding the tag (for example ‘1740’)

Extract the 4-digit SIC industry code from the annual report header (STANDARD INDUSTRIAL CLASSIFICATION).

Required: Evaluate whether adding the industry code as an additional tag improves the clustering. Use the standard deviation of profitability as a way to evaluate this. (Firms that are more similar, should have similar performance. Therefore, a better clustering would result in lower standard deviations for each cluster, relative to a worse clustering).

Do this for the filings for the year 2019 only. Calculate the standard deviation of performance for each cluster (use the year of CONFORMED END OF PERIOD, which are the first 4 digits of ‘date’ in summary.text).

For 50 clusters that means you will have 2 standard deviations for each cluster (one for each approach, with the extra SIC tag vs not adding the extra SIC tag). Use a t-test to test for a difference between the two sets of 50 standard deviations.

In [1]:
# all imports
import os as os
import pandas as pd
import csv
from pathlib import Path
import html, re
from w3lib.html import replace_entities
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.cluster import  hierarchy
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import gensim
stopWords = set(stopwords.words('english') )
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

/apps/jupyterhub/1.1.0/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from nltk.collocations import *

# table for 2019 filings with >=1000 characters, with links accordingly

In [3]:
df = pd.read_csv (r'ASG4.csv')  # did in the last assignment
mydirectory = '/blue/acg7849/hli1/item1'
newdirectory = '/blue/acg7849/share/BS/item1'

df["full_link"]= newdirectory +'/'+ df["fName"]
#print (sumdf[0:13]) 
listoflink = list(df["full_link"]) # in case we need a list
listofmylink = list(mydirectory +'/'+ df["fName"])

#print (listofmylink[0:5])
#print (df)

df.to_csv('ASG5.csv', index = True)

In [4]:
df = pd.read_csv (r'ASG5.csv')

In [5]:
type(df["length"])

pandas.core.series.Series

In [6]:
df["length"]= pd.to_numeric(df["length"])
# filter out all the things with <1000 length 
df = df.loc[df["length"] >= 1000]

In [7]:
print (df)

       Unnamed: 0  Unnamed: 0.1      CIK                 coName formtype  \
0               0             1  1289850      NeuroMetrix, Inc.     10-K   
1               1             2  1345016               YELP INC     10-K   
2               2             3  1245791      GI DYNAMICS, INC.     10-K   
3               3             4  1772177   KURA SUSHI USA, INC.     10-K   
4               4             5   832428        E.W. SCRIPPS Co     10-K   
...           ...           ...      ...                    ...      ...   
20848       20848         20849  1573221      TheRealReal, Inc.     10-K   
20849       20849         20850  1655008  HAHA GENERATION CORP.     10-K   
20850       20850         20851    54381             KAMAN Corp     10-K   
20851       20851         20852  1590976     Malibu Boats, Inc.     10-K   
20852       20852         20853  1419041            Tocagen Inc     10-K   

             date       fName  length                                full_link  
0     

In [8]:
df = df.iloc[: , 1:]
df = df.iloc[: , 1:]
print (df)

           CIK                 coName formtype        date       fName  \
0      1289850      NeuroMetrix, Inc.     10-K  20171231.0  267762.txt   
1      1345016               YELP INC     10-K  20191231.0  280603.txt   
2      1245791      GI DYNAMICS, INC.     10-K  20171231.0  267435.txt   
3      1772177   KURA SUSHI USA, INC.     10-K  20190831.0  275323.txt   
4       832428        E.W. SCRIPPS Co     10-K  20181231.0  277539.txt   
...        ...                    ...      ...         ...         ...   
20848  1573221      TheRealReal, Inc.     10-K  20191231.0  282124.txt   
20849  1655008  HAHA GENERATION CORP.     10-K  20191231.0  282912.txt   
20850    54381             KAMAN Corp     10-K  20171231.0  276068.txt   
20851  1590976     Malibu Boats, Inc.     10-K  20190630.0  272211.txt   
20852  1419041            Tocagen Inc     10-K  20171231.0  269435.txt   

       length                                full_link  
0      132616  /blue/acg7849/share/BS/item1/267762.txt

In [9]:
# saved as backup
#df.to_csv('ASG5.csv', index = False)

In [10]:
#filter the year 2019
df["date"]= pd.to_datetime(df["date"], format='%Y%m%d', errors='ignore')
#print(df["date"])
#print(df)
start_date = "2019-1-1"
end_date = "2019-12-31"

after_start_date = df["date"] >= start_date
before_end_date = df["date"] <= end_date
between_two_dates = after_start_date & before_end_date
df = df.loc[between_two_dates]

In [11]:
print (df)

           CIK                             coName formtype       date  \
1      1345016                           YELP INC     10-K 2019-12-31   
3      1772177               KURA SUSHI USA, INC.     10-K 2019-08-31   
8      1341766             Celsius Holdings, Inc.     10-K 2019-12-31   
9      1679826        Ping Identity Holding Corp.     10-K 2019-12-31   
12     1650445                 Quorum Health Corp     10-K 2019-12-31   
...        ...                                ...      ...        ...   
20843   922224                           PPL Corp     10-K 2019-12-31   
20846  1461755  Atlantic Capital Bancshares, Inc.     10-K 2019-12-31   
20848  1573221                  TheRealReal, Inc.     10-K 2019-12-31   
20849  1655008              HAHA GENERATION CORP.     10-K 2019-12-31   
20851  1590976                 Malibu Boats, Inc.     10-K 2019-06-30   

            fName  length                                full_link  
1      280603.txt  152323  /blue/acg7849/share/BS/item

In [12]:
# saved as backup
#df.to_csv('ASG5.csv', index = False)

# so we got: table for 2019 filings with >=1000 characters, with links accordingly

#### try generator and copy&paste

In [13]:
listoflink = list(df["full_link"]) # in case we need a list
print (listoflink[0:5])

['/blue/acg7849/share/BS/item1/280603.txt', '/blue/acg7849/share/BS/item1/275323.txt', '/blue/acg7849/share/BS/item1/280595.txt', '/blue/acg7849/share/BS/item1/283125.txt', '/blue/acg7849/share/BS/item1/282859.txt']


In [14]:
# generator function that returns one file at the time (just a string) - csv
# note that fit_transform expects one string for each file, so do not tokenize it
# this would be different for doc2vec, which expects a taggeddocument element

#files = 

def readBSGen():
    for i in listoflink[0:80]:                                                           
        with open ( i , encoding='utf-8') as b:
            BS = b.read()
        yield BS
# open the files with listed in csv ASG5

In [15]:
# set up vectorizer
vectorizer = TfidfVectorizer(stop_words='english')
# read documents using generator
tfidf = vectorizer.fit_transform( readBSGen(  ) )                # used readBSGen function and files w/yield
# dense
tfidf = tfidf.todense()                                   # this returns a matrix

In [16]:
print (tfidf)

[[0.         0.00466174 0.00678589 ... 0.         0.         0.        ]
 [0.0050993  0.02528593 0.         ... 0.         0.         0.        ]
 [0.         0.02800195 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.08022802 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.00563861 0.00978607 0.         ... 0.         0.         0.        ]]


In [17]:
import numpy as np
#c = np.savetxt('geekfile1.txt', tfidf, delimiter =', ') #1500 files = 3.4G large

In [18]:
len(listoflink)

4604

In [19]:
#try copy and paste targets - OK to run
import shutil

for i in range(len(listoflink)):
    #shutil.copy(item[0], "/blue/acg7849/hli1/item1")
    shutil.copy(listoflink[i], r'/blue/acg7849/hli1/item1')